In [230]:
import requests

In [231]:
apiEndPoints = {
    "album": r"https://api.deezer.com/album/{0}",
    "artist": r"https://api.deezer.com/artist/{0}",
    "track": r"https://api.deezer.com/track/{0}"    
}

In [232]:
access_token = "TOKEN"

In [233]:
def get_item_from_api(itemType, itemId):
    r = requests.get(url = apiEndPoints[itemType].format(itemId))
    return r.json()

In [234]:
get_item_from_api("artist", 27)

{'id': 27,
 'link': 'https://www.deezer.com/artist/27',
 'name': 'Daft Punk',
 'nb_album': 30,
 'nb_fan': 3029189,
 'picture': 'https://api.deezer.com/artist/27/image',
 'picture_big': 'https://cdns-images.dzcdn.net/images/artist/f2bc007e9133c946ac3c3907ddc5d2ea/500x500-000000-80-0-0.jpg',
 'picture_medium': 'https://cdns-images.dzcdn.net/images/artist/f2bc007e9133c946ac3c3907ddc5d2ea/250x250-000000-80-0-0.jpg',
 'picture_small': 'https://cdns-images.dzcdn.net/images/artist/f2bc007e9133c946ac3c3907ddc5d2ea/56x56-000000-80-0-0.jpg',
 'radio': True,
 'share': 'http://www.deezer.com/artist/27?utm_source=deezer&utm_content=artist-27&utm_term=0_1454845793&utm_medium=web',
 'tracklist': 'https://api.deezer.com/artist/27/top?limit=50',
 'type': 'artist'}

In [235]:
def get_history(processingFunction = process_history, length = 50):
    output = []
    increment = 50
    numberOfSteps = length // increment
    for step in range(numberOfSteps):
        url = r"http://api.deezer.com/user/me/history?access_token={0}&index={1}&limit={2}".format(access_token,step*increment, increment)
        raw = requests.get(url = url).json()
        output.extend(processingFunction(raw))
    return output

In [ ]:
get_history()

In [ ]:
from collections import namedtuple

In [ ]:
Listen = namedtuple('Listen', ['song_name', 'song_id', 'album_id', 'artist_id', "timestamp"], verbose=True)

In [ ]:
example = Listen("Defector", "Drones", "Muse", 1)

In [ ]:
raw = get_history()

In [ ]:
def process_history(data):
    return [Listen(item["title"], item["id"], item["album"]["id"], item["artist"]["id"], item["timestamp"]) for item in data["data"]]

In [ ]:
longHistory = get_history(length = 1000)

In [ ]:
listenHistory = process_history(historyExample)

In [ ]:
import numpy as np
%matplotlib inline
from IPython.core.pylabtools import figsize
from matplotlib import pyplot as plt
from collections import Counter

In [ ]:
timestamps = [x.timestamp for x in longHistory]
artists = [x.artist_name for x in longHistory]

In [ ]:
def display_pie_chart(listenHistory, number_items = 10):
    artists = [x.artist_name for x in listenHistory]
    counter = Counter(artists)
    
    artists, count = [x[0] for x in counter.most_common(number_items)], [x[1] for x in counter.most_common(number_items)]
    plt.pie(count, labels = artists)

In [ ]:
display_pie_chart(longHistory, number_items = 20 )

In [ ]:
def most_listened_to_songs(listenHistory, number_items = 40):
    titles = [x.song_id for x in listenHistory]
    counter = Counter(titles)
    return [x[0] for x in counter.most_common(number_items)]

In [ ]:
most_listened_to_songs(longHistory)

In [ ]:
class Playlist:
    
    def __init__(self, p_id = 1609467705,name = "JakesSuperAmazingFlow"):
        self.name = name
        self.url = r"http://api.deezer.com/playlist/{0}/tracks?access_token={1}".format(p_id,access_token)
        self.p_id = p_id

    def create(self):
        return False
        
    def add_tracks(self, ids):
        url = self.url + "&songs=" + self._parse_track_input(ids)
        return requests.post(url).json()
    
    def get_tracks(self):
        return requests.get(self.url).json()
    
    def remove_tracks(self, ids):
        url = self.url + "&songs=" + self._parse_track_input(ids)
        requests.delete(url)
        
    def clear_tracks(self):
        tracks = [x["id"] for x in playlist.get_tracks()["data"]]
        while tracks != []:
            self.remove_tracks(tracks)
            tracks = [x["id"] for x in playlist.get_tracks()["data"]]
    
    
    def _parse_track_input(self, ids):
        return "%2C".join([str(id) for id in ids])
    

In [ ]:
import random

In [228]:
def select_from_generator(songList, amount):
    if len(songList) < amount: return songList
    random.shuffle(songList)
    return songList[:amount - 1]

def generate_flow(generators, history = longHistory, weights = None):
    
    flow_length = 50
    
    if weights == None: weights = [1] * len(generators)
        
    totalWeight = sum(weights)
        
    selectPerGenerator = [int(flow_length * (weight / totalWeight)) for weight in weights ] 
    
    print(selectPerGenerator)
    
    output = []
    
    for i in range(len(generators)):
        output.extend(select_from_generator(generators[i](history), selectPerGenerator[i]))
        
    
    return output

In [238]:
playlist = Playlist()
playlist.clear_tracks()
playlist.add_tracks(generate_flow([most_listened_to_songs, random_song_from_top_n_artists, deezer_flow]))

[16, 16, 16]
Counter({'Go Your Own Way (2004 Remastered Edition)': 21, 'We Are the People': 20, 'Girl From Mars': 16, 'Dreams (Remastered)': 14, 'Clarity (Radio Edit Version)': 14, 'Everywhere': 13, 'The Chain (Remastered)': 13, 'Mercy': 11, 'Second Hand News (2004 Remastered Edition)': 11, 'Little Lies': 11, 'Dead Inside': 11, 'Never Going Back Again (2004 Remastered Edition)': 10, 'Reapers': 9, 'Psycho': 9, 'Somebody Told Me': 9, 'Dreams (2004 Remastered Edition)': 9, 'Some Might Say': 9, 'Aftermath': 8, 'Young Folks': 8, 'Teardrop': 8, "Don't Stop (2004 Remastered Edition)": 8, 'Tokyo (Vampires & Wolves)': 7, 'Defector': 7, '[JFK]': 7, 'The Handler': 7, 'Songbird': 6, 'Best Day Of My Life': 6, 'Sonnet': 6, 'In The Heat Of The Moment': 6, 'Are You Gonna Be My Girl': 6, 'How You Remind Me': 6, 'The Chain (2004 Remastered Edition)': 6, 'Revolt': 6, 'Princess of China': 6, 'Doom And Gloom': 6, 'Kids': 6, 'Golden Skans': 6, 'Just': 6, "I Don't Want To Know (2004 Remastered Edition)": 5, 

True

In [236]:
generate_flow([most_listened_to_songs, random_song_from_top_n_artists, deezer_flow])

[16, 16, 16]
Counter({'Go Your Own Way (2004 Remastered Edition)': 21, 'We Are the People': 20, 'Girl From Mars': 16, 'Dreams (Remastered)': 14, 'Clarity (Radio Edit Version)': 14, 'Everywhere': 13, 'The Chain (Remastered)': 13, 'Mercy': 11, 'Second Hand News (2004 Remastered Edition)': 11, 'Little Lies': 11, 'Dead Inside': 11, 'Never Going Back Again (2004 Remastered Edition)': 10, 'Reapers': 9, 'Psycho': 9, 'Somebody Told Me': 9, 'Dreams (2004 Remastered Edition)': 9, 'Some Might Say': 9, 'Aftermath': 8, 'Young Folks': 8, 'Teardrop': 8, "Don't Stop (2004 Remastered Edition)": 8, 'Tokyo (Vampires & Wolves)': 7, 'Defector': 7, '[JFK]': 7, 'The Handler': 7, 'Songbird': 6, 'Best Day Of My Life': 6, 'Sonnet': 6, 'In The Heat Of The Moment': 6, 'Are You Gonna Be My Girl': 6, 'How You Remind Me': 6, 'The Chain (2004 Remastered Edition)': 6, 'Revolt': 6, 'Princess of China': 6, 'Doom And Gloom': 6, 'Kids': 6, 'Golden Skans': 6, 'Just': 6, "I Don't Want To Know (2004 Remastered Edition)": 5, 

[('Girl From Mars', 16),
 ('Kids', 6),
 ('Doom And Gloom', 6),
 ('Dreams (2004 Remastered Edition)', 9),
 ('Best Day Of My Life', 6),
 ('How You Remind Me', 6),
 ("Don't Stop (2004 Remastered Edition)", 8),
 ('Young Folks', 8),
 ('Some Might Say', 9),
 ('Tokyo (Vampires & Wolves)', 7),
 ('Second Hand News (2004 Remastered Edition)', 11),
 ("I Don't Want To Know (2004 Remastered Edition)", 5),
 ('Princess of China', 6),
 ('Dead Inside', 11),
 ('In The Heat Of The Moment', 6),
 101215858,
 63480990,
 101215850,
 74172985,
 70322139,
 3599714,
 26608101,
 67549275,
 664437,
 26608061,
 70322134,
 85909191,
 70322133,
 664445,
 4315310,
 946317,
 3165194,
 941558,
 30521631,
 102292706,
 3134034,
 3157501,
 3751638,
 55829181,
 69975639,
 13181907,
 1093484,
 7372874,
 68097788,
 2541626]

In [216]:
def random_song_from_top_n_artists(history, number_of_artists = 5):
    artists = [x.artist_id for x in history]
    counter = Counter(artists)
    artist_ids = [x[0] for x in counter.most_common(5)]
    songs = []
    for artist_id in artist_ids:
        raw = requests.get(r"https://api.deezer.com/artist/{0}/top?limit=20".format(artist_id)).json()
        songs.extend([x["id"] for x in raw["data"]])
    return songs

In [224]:
def deezer_flow(history):
    url = r"http://api.deezer.com/user/me/flow?access_token={0}".format(access_token)
    return [x["id"] for x in requests.get(url).json()["data"]]

In [225]:
deezer_flow("Jake")

[15908435,
 2420458,
 95956384,
 13139232,
 910079,
 74683990,
 1178034,
 3129407,
 4622567,
 3157501,
 3590194,
 13547128,
 6685848,
 2924841,
 4624027,
 946317,
 3130433,
 2754843,
 102292706,
 44230981,
 1178161,
 67238732,
 39376531,
 2936707,
 7179758]